Custom atmospheric profiles
===========================

.. admonition:: Overview

   This tutorials intoduces Eradiate's capabilities to handle customized atmospheric profiles beyond classical rescaling.

.. admonition:: Prerequisites

   The reader should familiarize themselves with the user guide about Eradiate terminology and the `one-dimensional experiment features`. Additionally users should be familiar with the `molecular atmosphere basics` tutorial.

.. admonition:: What you will learn

   This tutorial will teach you how to format a custom atmospheric profile for Eradiate and how to use it in a `MolecularAtmosphere`.

.. admonition:: Additional requirements

   An example atmosphere profile is available at `<PATH>` in the `eradiate-data` repository

----

## Standard profiles and rescaling

First, let us refresh the basics of standard atmosphere profiles and classic rescaling in Eradiate.

Eradiate uses the `joseki` package for handling atmosphere profiles.

We import a few necessary packages and create two molecular atmospheres. One with a basic profile and one with a rescaled molecule.

In [ ]:
%load_ext eradiate
import eradiate
import joseki
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from eradiate.scenes.atmosphere import MolecularAtmosphere
from eradiate import unit_registry as ureg

In [ ]:
eradiate.set_mode("ckd")

In [ ]:
us_standard = MolecularAtmosphere(
    thermoprops={
        "identifier": "afgl_1986-us_standard",
        "z": np.linspace(0.0, 120.0, 121) * ureg.km,
    },
)

If you recall the `molecular atmosphere basics` tutorial, you will recognize this workflow. We have now created a molecular atmosphere, using the AFGL 1986 US-Standard atmosphere profile.

Next, let us create another atmosphere with a rescaled amount of carbon dioxide.

In [ ]:
# First, create the original standard profile:
thermoprops = joseki.make(
    identifier="afgl_1986-us_standard",
    z=np.linspace(0.0, 120.0, 121) * ureg.km,
)

# Then, rescale it:
rescaled = thermoprops.joseki.rescale(
    factors={
        "O3": 1.25,
    }
)

# Finally instanciate the MolecularAtmosphere class with the rescaled
# thermophysical profile
molecular_rescaled = MolecularAtmosphere(
    thermoprops=rescaled
)

Now we can plot the vertical profile of ozone and compare both atmospheres.

In [ ]:
us_standard._thermoprops.x_O3.plot(y="z", label="US-Standard")
molecular_rescaled._thermoprops.x_O3.plot(y="z", label="Rescaled to 125%")
plt.legend()

## Custom profiles

The fundamental limitation of classic rescaling is that it only controls the total integrated concentration of an atmospheric constituent. The detailed vertical distribution can not be changed. Eradiate adresses this problem by allowing users to add fully custom distributions of constituent gasses in the `MolecularAtmosphere` object.

Essentially, the requirements for a custom atmosphere profile are the same as for the `thermoprops` object, we defined above:

In [ ]:
thermoprops

- Thermophysical properties must have only one dimension, named "z", which describes the vertical grid. This vertical grid must have regular spacing. In the example, level range from 0 to 120 kilometers and are 1 kilometer apart
- The data variables p, t, and n, which represent the atmospheric pressure, temperature and air number density, must be present to define the vertical profile of the atmosphere.
- The molar fractions of all gases which are available in Eradiate's absorption data (data variables whose names start with `x_`) should be present, but in its default configuration, Eradiate will not raise an Error if a molecule is missing.

**The absorption by molecules which are not present in the thermophysical properties dataset will not be taken into account in the simulations!**